### Connecting to Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd drive
%cd MyDrive
!dir

/content/drive
/content/drive/MyDrive
abhi\ (1).jpg  Colab\ Notebooks		       plant_disease_dataset
Blender        new-plant-diseases-dataset.zip  Plant\ Disease\ Dataset


### Installing Kaggle API

In [ ]:
!pip install kaggle

### Uploadding Kaggle API key

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


### Configuring the Kaggle API

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

### Downloading the Dataset

In [ ]:
!kaggle datasets download -d vipoooool/new-plant-diseases-dataset

new-plant-diseases-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!dir
!cd drive
!dir

abhi\ (1).jpg	  kaggle.json			  Plant\ Disease\ Dataset
Blender		  new-plant-diseases-dataset.zip
Colab\ Notebooks  plant_disease_dataset
/bin/bash: line 1: cd: drive: No such file or directory
abhi\ (1).jpg	  kaggle.json			  Plant\ Disease\ Dataset
Blender		  new-plant-diseases-dataset.zip
Colab\ Notebooks  plant_disease_dataset


### Unziping the Dataset

In [8]:
!unzip /content/drive/MyDrive/new-plant-diseases-dataset.zip -d /content/drive/MyDrive/plant_disease_dataset

Streaming output truncated to the last 5000 lines.
  inflating: /content/drive/MyDrive/plant_disease_dataset/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/valid/Strawberry___healthy/273a7a9e-18be-4b6a-976a-fa5ffd69b731___RS_HL 4366_90deg.JPG  
  inflating: /content/drive/MyDrive/plant_disease_dataset/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/valid/Strawberry___healthy/275f8963-f4f4-4903-962b-1da716725d08___RS_HL 4780_90deg.JPG  
  inflating: /content/drive/MyDrive/plant_disease_dataset/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/valid/Strawberry___healthy/275f8963-f4f4-4903-962b-1da716725d08___RS_HL 4780_new30degFlipLR.JPG  
  inflating: /content/drive/MyDrive/plant_disease_dataset/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/valid/Strawberry___healthy/27c2aaa4-de4b-4fb1-ba8d-14d4b24b2afa___RS_HL 4665.JPG  
  inflating: /content/drive/MyDrive/plant_disease_data

### Loading the Dataset

In [9]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import os

# Define data transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224,0.225])
    ]),
    'valid':transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224,0.225])
    ])
}

# create datasets
data_dir = "/content/drive/MyDrive/plant_disease_dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)"
image_datasets = {x: ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'valid']}

# Create dataloaders
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=4) for x in ['train', 'valid']}

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


### Device agnostic code

In [10]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### Loading Pretrained model

In [11]:
import torch
import torch.nn as nn
import torchvision.models as models

# Load a pre-trained model
model = models.resnet18(pretrained=True)

# Modify the output layer for your specific task
num_classes = len(image_datasets['train'].classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


### Training Loop

In [12]:
import torch.optim as optim

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
def train_model(model, dataloaders, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')


### Training the model

In [ ]:
# Train the model
model = model.to(device)
train_model(model, dataloaders, criterion, optimizer, num_epochs=10)


train Loss: 0.3001 Acc: 0.9053
valid Loss: 0.0969 Acc: 0.9693
train Loss: 0.2257 Acc: 0.9287
valid Loss: 0.1006 Acc: 0.9676
train Loss: 0.1851 Acc: 0.9415
valid Loss: 0.0596 Acc: 0.9805
train Loss: 0.1687 Acc: 0.9464
valid Loss: 0.0932 Acc: 0.9734
train Loss: 0.1503 Acc: 0.9515
valid Loss: 0.0513 Acc: 0.9820
train Loss: 0.1369 Acc: 0.9558
valid Loss: 0.0497 Acc: 0.9840
train Loss: 0.1230 Acc: 0.9609
valid Loss: 0.0469 Acc: 0.9854
train Loss: 0.1166 Acc: 0.9624
valid Loss: 0.0284 Acc: 0.9902
train Loss: 0.1113 Acc: 0.9641
valid Loss: 0.0442 Acc: 0.9860
train Loss: 0.1053 Acc: 0.9655
valid Loss: 0.0354 Acc: 0.9889


KeyError: ignored

In [ ]:
image_datasets

{'train': Dataset ImageFolder
     Number of datapoints: 70295
     Root location: /content/drive/MyDrive/plant_disease_dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train
     StandardTransform
 Transform: Compose(
                RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear, antialias=warn)
                RandomHorizontalFlip(p=0.5)
                ToTensor()
                Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ),
 'valid': Dataset ImageFolder
     Number of datapoints: 17572
     Root location: /content/drive/MyDrive/plant_disease_dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid
     StandardTransform
 Transform: Compose(
                Resize(size=256, interpolation=bilinear, max_size=None, antialias=warn)
                CenterCrop(size=(224, 224))
                ToTensor()
                Normalize(mean=[0.

### Saving the model

In [ ]:
import torch

# Assuming 'model' is your trained PyTorch model
# Save the model to a file
torch.save(model.state_dict(), 'model.pth')

### Loading the model

In [17]:
import torch
from torchvision import transforms
from PIL import Image
from torchvision import models

# Create an instance of the pretrained model
model = models.resnet18(pretrained=True)

num_classes = len(image_datasets['train'].classes)  # Replace with the number of classes in your task
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)


In [22]:
model.load_state_dict(torch.load('model.pth'))
model.eval()  # Set the model to evaluation mode

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [72]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder

# Define the data transformations for test data
test_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define the path to your test dataset
test_data_dir = "/content/drive/MyDrive/plant_disease_dataset/test"

# Create a test dataset using ImageFolder
test_dataset = ImageFolder(root=test_data_dir, transform=test_transforms)

# Create a DataLoader for the test dataset
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [73]:
correct = 0
total = 0

# Set the model to evaluation mode
model.eval()
model.to(device)

# Iterate over the test dataset and evaluate the model
with torch.no_grad():
    for inputs, labels in test_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)  # Get the index of the max logit as the predicted class
        total += labels.size(0)  # Increment the total number of samples
        correct += (predicted == labels).sum().item()  # Increment the correct predictions count

# Calculate the accuracy
accuracy = (correct / total) * 100
print(f'Test Accuracy: {accuracy:.2f}%')


Test Accuracy: 9.09%


In [77]:
predicted,labels

(tensor([35], device='cuda:0'), tensor([0], device='cuda:0'))

### Making prediction

In [52]:
class_label = {
	0 : 'Apple scab',
	1 : 'Apple black rot',
	2 : 'Apple cedar apple rust',
	3 : 'Apple healthy',
	4 : 'Blueberry healthy',
	5 : 'Cherry powdery mildew',
	6 : 'Cherry healthy',
	7 : 'Corn Cercospora leaf Gray leaf spot',
	8 : 'Corn common rust',
	9 : 'Corn Northen leaf blight',
	10 : 'Corn healthy',
	11 : 'Grap black rot',
	12 : 'Grap esca (black measles)',
	13 : 'Grap leaf blight (Isaropsis leaf spot)',
	14 : 'Grap healthy',
	15 : 'Orange Haunglonbing (Citrus greening)',
	16 : 'Peach Bacterial spot',
	17 : 'Peach healthy',
	18 : 'Pepper bell backterial spot',
	19 : 'Pepper bell healthy',
	20 : 'Potato early blight',
	21 : 'Potato late blight',
	22 : 'Potato healthy',
	23 : 'Raspberry healthy',
	24 : 'Soybean healthy',
	25 : 'Squash Powdery mildew',
	26 : 'Strawberry leaf scorch',
	27 : 'Strawberry healthy',
	28 : 'Tomato bacterial spot',
	29 : 'Tomato early blight',
	30 : 'Tomato late blight',
	31 : 'Tomato leaf mold',
	32 : 'Tomato septoria leaf spot',
	33 : 'Tomato spider mites two spotted spider mite',
	34 : 'Tomato target spot',
	35 : 'Tomato yellow leaf curl virus',
	36 : 'Tomato mosaic virus',
	37 : 'Tomato healthy'
}

### Predicting the disease

In [75]:
# Load and preprocess your input image
image_path = "/content/drive/MyDrive/plant_disease_dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/Corn_(maize)___Northern_Leaf_Blight/005318c8-a5fa-4420-843b-23bdda7322c2___RS_NLB 3853 copy.jpg"
image = Image.open(image_path)  # Replace 'input_image.jpg' with the path to your image
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
input_data = transform(image).unsqueeze(0)  # Add a batch dimension

input_data = input_data.to(device)
with torch.no_grad():
    output = model(input_data)

# Interpret the predictions based on your specific task (e.g., classification)
_, predicted_class = torch.max(output, 1)

# If your task involves multiple classes, you can use the output probabilities as well
probabilities = torch.nn.functional.softmax(output, dim=1)

predicted_label = class_label[predicted_class.item()]

print(f"Prediction : {predicted_label}")

Prediction : Corn Northen leaf blight
